<a href="https://colab.research.google.com/github/GarlandZhang/hairy_gan/blob/master/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing import image as kp_image

from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras import backend as K

from PIL import Image

import numpy as np

import os

Using TensorFlow backend.


In [2]:
# list of layers to caulcate for content and style loss
content_layers = ['block3_conv3']
style_layers = ['block1_conv1', 'block2_conv2', 'block4_conv3']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

project_path = '/content/drive/My Drive/hairy_gan'
content_img_path = os.path.join(project_path, 'content.jpg')
style_img_path = os.path.join(project_path, 'style.jpg')
save_img_path = os.path.join(project_path, 'generated.jpg')

vgg_weights_path = os.path.join(project_path, 'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [3]:
# preprocess image
def load_img(img_path):
  img = Image.open(img_path)

  # resize to max dimension
  max_dim = 512
  img_size = max(img.size)
  scale = max_dim / img_size 
  img = img.resize((round(img.size[0] * scale), round(img.size[1] * scale)), Image.ANTIALIAS)

  img = np.expand_dims(img, axis=0)

  # required step to run vgg19
  out = tf.keras.applications.vgg19.preprocess_input(img)

  return tf.convert_to_tensor(out)

# postprocess image
def postprocess_img(processed_img):
  img = processed_img.copy()

  img[:, :, 0] += 103.939
  img[:, :, 1] += 116.779
  img[:, :, 2] += 123.68

  img = img[:, :, ::-1]

  img = np.clip(img, 0, 255).astype('uint8')

  return img

In [4]:
# build loss functions from scratch

def gram_matrix(input_tensor):
  # 3D => 2D matrix: nh * nw * nc => nc * (nh * nw)
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]

  gram = tf.matmul(a, a, transpose_a=True)

  return gram

def get_style_loss(base_style, gram_target): # base_style is generated layer output, gram_target is style layer output
  height, width, channels = base_style.get_shape().as_list()

  gram_style = gram_matrix(base_style)

  return tf.reduce_mean(tf.square(gram_style - gram_target)) / (channels**2 * width * height) #(4.0 * (channels ** 2) * (width * height) ** 2)

def get_content_loss(content, target): # content is new generated image, target is original image
  return tf.reduce_mean(tf.square(content - target)) / 2


def compute_loss(model, loss_weights, generated_output_activations, gram_style_features, content_features, num_content_layers, num_style_layers):
  gen_style_features, gen_content_features = generated_output_activations

  style_weight, content_weight = loss_weights

  style_score = 0
  content_score = 0

  # accumulate style losses from all layers
  # weight the same fore ach layer
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, gen_style_features):
    style_score = style_score + weight_per_style_layer * get_style_loss(comb_style[0], target_style) 

  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, gen_content_features):
    content_score = content_score + weight_per_content_layer * get_content_loss(comb_content[0], target_content)

  loss = style_weight * style_score + content_weight * content_score

  return loss, style_score, content_score

In [5]:
# get feature outputs for style and content images
def get_feature_representations(model, content_img_path, style_img_path, num_content_layers):

  content_img = load_img(content_img_path)
  style_img = load_img(style_img_path)

  content_outputs, _ = model(content_img) # only get content features of content image (dont care about style)
  _, style_outputs = model(style_img) # only get style features of style image

  content_features = [ content_layer[0] for content_layer in content_outputs ]
  style_features = [ style_layer[0] for style_layer in style_outputs ]

  return style_features, content_features

In [6]:
def build_model(content_layers, style_layers):
  vgg19 = VGG19(weights=None, include_top=False)
  vgg19.trainable = False # pretrained; don't touch

  content_model_outputs = [vgg19.get_layer(layer).output for layer in content_layers]
  style_model_outputs = [vgg19.get_layer(layer).output for layer in style_layers]

  return Model(inputs=vgg19.input, outputs=[content_model_outputs, style_model_outputs]), vgg19 # takes part of model up to the last output 

In [22]:
def run_style_transfer(content_img_path, style_img_path, save_img_path, num_content_layers, num_style_layers, num_iterations=200, content_weight=0.1, style_weight=0.9):
  sess = tf.compat.v1.Session()
  tf.compat.v1.keras.backend.set_session(sess)

  model, vgg19 = build_model(content_layers, style_layers)

  # get layer outputs from the content image and style image
  style_features, content_features = get_feature_representations(model, content_img_path, style_img_path, num_content_layers)

  # get gram matrix values to prepare for total loss calc
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]

  # get layer outputs for generated image
  generated_image = load_img(content_img_path)
  generated_image = tf.Variable(generated_image, dtype=tf.float32)
  model_outputs = model(generated_image)

  loss_weights = (style_weight, content_weight)

  loss = compute_loss(model, loss_weights, model_outputs, gram_style_features, content_features, num_content_layers, num_style_layers)
  optimizer = tf.train.AdamOptimizer(learning_rate=9, beta1=0.9, epsilon=.1).minimize(loss[0], var_list=[generated_image]) # why do I need generated image?

  sess.run(tf.global_variables_initializer())
  sess.run(generated_image.initializer)

  vgg19.load_weights(vgg_weights)

  # the loss is only updated if the current loss is better (i.e lesser) than the previous loss
  best_loss, best_img = None, None

  # VGG default normalization
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means

  for i in range(num_iterations):
    sess.run(opt) # where does sess.run store these values? what does it do with them?

    # keeps image values in range of min, max values 
    clipped = tf.clip_by_value(generated_image, min_vals, max_vals)

    # assign the clipped value to the tensor stylized image
    generated_image.assign(clipped)

    total_loss, style_score, content_score = loss
    total_loss = total_loss.eval(session=sess)

    if best_loss == None or total_loss < best_loss:
      best_loss = total_loss

      temp_gen_image = sess.run(generated_image)[0]
      best_img = postprocess_img(temp_gen_image)

      s_loss = sess.run(style_score)
      c_loss = sess.run(content_score)

      print(f'best: iteration: {i} loss: {total_loss}, style_loss: {s_loss}, content_loss: {c_loss}')

    if (i + 1) % 100 == 0:
      output = Image.fromarray(best_img)
      output.save(os.path.join(project_path, f'{i + 1}-{save_img_path}'))

  sess.close()

  return best_img, best_loss

In [23]:
best, best_loss = run_style_transfer(content_img_path, style_img_path, save_img_path, num_content_layers, num_style_layers)

ValueError: ignored

AttributeError: ignored